<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Explore-other-reward-functions" data-toc-modified-id="Explore-other-reward-functions-1">Explore other reward functions</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Code" data-toc-modified-id="Code-1.3">Code</a></span><ul class="toc-item"><li><span><a href="#Data-generation" data-toc-modified-id="Data-generation-1.3.1">Data generation</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1.3.2">Evaluation</a></span></li><li><span><a href="#Visualization" data-toc-modified-id="Visualization-1.3.3">Visualization</a></span></li></ul></li><li><span><a href="#Visualizing-games" data-toc-modified-id="Visualizing-games-1.4">Visualizing games</a></span><ul class="toc-item"><li><span><a href="#07_x64_discount_factor_1" data-toc-modified-id="07_x64_discount_factor_1-1.4.1">07_x64_discount_factor_1</a></span></li></ul></li><li><span><a href="#Populating-the-agent-pool" data-toc-modified-id="Populating-the-agent-pool-1.5">Populating the agent pool</a></span><ul class="toc-item"><li><span><a href="#Adding-new-agents" data-toc-modified-id="Adding-new-agents-1.5.1">Adding new agents</a></span></li><li><span><a href="#Does-having-more-good-agents-stabilize-scores?" data-toc-modified-id="Does-having-more-good-agents-stabilize-scores?-1.5.2">Does having more good agents stabilize scores?</a></span></li></ul></li><li><span><a href="#Useful-snippets" data-toc-modified-id="Useful-snippets-1.6">Useful snippets</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.7">Summary</a></span></li></ul></li></ul></div>

# Explore other reward functions

## Goal

The goal of this notebooks is to continue learning about the Q value function, but trying other reward methods.

This is a continuation of notebook 004, which had become too big. In this notebook I will split the functionality
of playing games and generating data from training from those games.

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import kaggle_environments
from kaggle_environments import make
from kaggle_environments import evaluate
from kaggle_environments.envs.hungry_geese.hungry_geese import Action
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import pandas as pd
from tqdm.notebook import tqdm
import time
from itertools import permutations
from functools import partial
import gc
from concurrent.futures import ProcessPoolExecutor
import json
import logging
import glob
from IPython.display import clear_output
import tempfile
import yaml
import seaborn as sns

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
#os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import tensorflow as tf
import tensorflow.keras as keras

from hungry_geese import GameState
from hungry_geese.state import make_board_egocentric, get_head_position, combine_data, horizontal_simmetry, vertical_simmetry
from hungry_geese.utils import ACTIONS, opposite_action, get_timestamp, log_ram_usage, configure_logging
from hungry_geese.definitions import ACTION_TO_IDX
from hungry_geese.agents import EpsilonAgent, QValueAgent, SoftmaxAgent, QValueSafeAgent
from hungry_geese.evaluation import play_matches_in_parallel, monitor_progress
from hungry_geese.elo import EloRanking
from hungry_geese.model import simple_model, create_model_for_training

%config Completer.use_jedi = False

Loading environment football failed: No module named 'gfootball'


In [3]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (30, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

configure_logging(logging.INFO)
logger = logging.getLogger(__name__)

## Code

### Data generation

In [4]:
def play_matches_in_parallel_and_save_history(agents, max_workers=20, n_matches=1000,
                                              output_path=None,
                                              running_on_notebook=True):
    """
    Plays n_matches in parallel using ProcessPoolExecutor

    Parameters
    -----------
    agents : list
        List of the agents that we will use for playing
    """
    log_ram_usage()
    output_path = output_path or get_games_output_path(agents, n_matches)
    assert not os.path.exists(output_path), 'Output path already exists: %s' % output_path
    with ProcessPoolExecutor(max_workers=max_workers) as pool:
        matches_results = []
        submits = []
        for i in range(n_matches):
            if isinstance(agents, list):
                submits.append(pool.submit(play_game, agents=agents))
            elif callable(agents):
                submits.append(pool.submit(play_game, agents=agents()))
            else:
                raise TypeError(type(agents))
        monitor_progress(submits, running_on_notebook)
        matches_results = [submit.result() for submit in submits]
    
    log_ram_usage()
    logger.info('saving matches to json: %s' % output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(matches_results, f)
        
    del submits
    del matches_results
    gc.collect()
    log_ram_usage()

def play_game(agents):
    env = make("hungry_geese")
    return env.run(agents=agents)

def get_games_output_path(agents, n_matches):
    output_path = '/mnt/hdd0/Kaggle/hungry_geese/games/%s_vs_%s_matches_%i.json' % (
        os.path.splitext(os.path.basename(agents[0]))[0], 
        os.path.splitext(os.path.basename(agents[1]))[0], 
        n_matches)
    return output_path

In [5]:
def test_agent(agent):
    print('Testing agent: %s' % agent)
    env = make('hungry_geese', debug=True)
    env.run([agent]*4)

In [6]:
def create_train_data_from_saved_matches(saved_games_paths, reward_name, output_path, agent_idx_range=None):
    """
    Creates train data without any simmetry
    
    Parameters
    ----------
    saved_games_paths : list of str
        Path to the games that we want to use
    reward_name : str
        Name of the reward function that we want to use
    output_path : str
        Path to the file were we are going to save the results
    max_workers : int
    agent_idx_range : list of int
        Idx of the agents we want to use for collecting data, if None all the agents
        will be used
    """
    env = make("hungry_geese")
    conf = env.configuration

    state = GameState(reward_name=reward_name)
    train_data = []
    agent_idx_range = agent_idx_range or list(range(4))
    
    for saved_games_path in tqdm(saved_games_paths, desc='looping over saved games'):
        log_ram_usage()
        with open(saved_games_path, 'r') as f:
            matches_results = json.load(f)
        log_ram_usage()
    
        for _ in tqdm(range(len(matches_results)), desc='Creating game data'):
            match = matches_results[0]
            for idx in agent_idx_range:
                state.reset()
                for step_idx, step in enumerate(match):
                    observation = step[0]['observation'].copy()
                    observation['index'] = idx
                    state.update(observation, conf)
                    if step_idx:
                        state.add_action(step[idx]['action'])
                    if not observation['geese'][idx]:
                        break
                train_data.append(state.prepare_data_for_training())
            del matches_results[0]
        
    log_ram_usage()
    logger.info('Going to combine the data')
    train_data = combine_data(train_data)
    log_ram_usage()
    logger.info('Size of the boards is %.1f GB (%s [%.1f GB])' % (
        train_data[0].nbytes/1e9,
        str([round(data.nbytes/1e9, 1) for data in train_data]),
        np.sum([data.nbytes/1e9 for data in train_data])))
    logger.info('Saving data on: %s' % output_path)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    np.savez_compressed(output_path, boards=train_data[0], features=train_data[1], actions=train_data[2], rewards=train_data[3])
    del state
    del train_data
    gc.collect()
    log_ram_usage()

In [7]:
def create_train_data(agents, n_matches, reward_name, agent_idx_range=None):
    """
    Creates train data without any simmetry
    
    Parameters
    ----------
    agents : list of str
        Names or paths of the agents
    n_matches : int
        Number of matches to play
    reward_name : str
        Name of the reward function that we want to use
    max_workers : int
    agent_idx_range : list of int
        Idx of the agents we want to use for collecting data, if None all the agents
        will be used
    """
    saved_games_path = get_games_output_path(agents, n_matches)
    if not os.path.exists(saved_games_path):
        play_matches_in_parallel_and_save_history(agents, n_matches=n_matches)

    output_path = get_data_output_path(agents, n_matches, reward_name)
    create_train_data_from_saved_matches([saved_games_path], reward_name, output_path, agent_idx_range=agent_idx_range)
    
def get_data_output_path(agents, n_matches, reward_name):
    output_file = '/mnt/hdd0/Kaggle/hungry_geese/data/%s/%s_vs_%s_matches_%i.npz' % (
        reward_name,
        os.path.splitext(os.path.basename(agents[0]))[0], 
        os.path.splitext(os.path.basename(agents[1]))[0], 
        n_matches)
    return output_file

### Evaluation

In [8]:
initial_elo_ranking = pd.read_csv('../data/elo_ranking.csv', index_col='model')
initial_elo_ranking

,ranking
model,
icedragon,1602
iceberg,1548
wallbreaker,1515
ram,1463
quantum,1398
clasic,1365
boilergoose,1269
besthoarder,1227
crazy_goose,1186


In [9]:
with open('../data/agents.yml', 'r') as f:
    agents = yaml.safe_load(f)

In [10]:
def evaluate_agent(new_agent, adversary_agents, n_matches, single_agent=True, max_workers=20, run_agent_test=False):
    agent_name = list(new_agent.keys())[0]
    # I have found that for certain agents if the test is run then evaluation is frozen
    if run_agent_test: test_agent(new_agent[agent_name])
    reduced_agents_set = new_agent.copy()
    for adversary_agent in adversary_agents:
        reduced_agents_set[adversary_agent] = agents[adversary_agent]
    
    if single_agent:
        sample_agents_func = lambda: [agent_name] + np.random.choice(adversary_agents, 3, replace=False).tolist()
    else:
        def sample_agents_func():
            while 1:
                sampled_agents = [agent_name] + np.random.choice(adversary_agents + [agent_name], 3).tolist()
                if len(np.unique(sampled_agents)) >=2:
                    break
            return sampled_agents

    matches_agents, matches_results = play_matches_in_parallel(reduced_agents_set, sample_agents_func, n_matches=n_matches, max_workers=max_workers)
    return matches_agents, matches_results, reduced_agents_set
    
def compute_elo_ranking(matches_agents, matches_results, reduced_agents_set):
    initial_agent_elo = 1000
    for k in [32, 16, 8, 4, 2, 1]:
        agent_name = [name for name in reduced_agents_set if name not in initial_elo_ranking][0]
        initial_ranking = initial_elo_ranking.to_dict()['ranking']
        initial_ranking[agent_name] = initial_agent_elo
        initial_ranking = {key: initial_ranking[key] for key in reduced_agents_set}
        elo_ranking = EloRanking(initial_ranking, {agent_name}, k=k)
        for match_agents, match_results in zip(matches_agents, matches_results):
            elo_ranking.add_match(match_agents, match_results)
        initial_agent_elo = elo_ranking.summary().loc[agent_name, 'ranking']
    elo_ranking.plot()
    return elo_ranking.summary()

In [11]:
def simple_agent_evaluation(agent_path, n_matches=500):
    """
    Computes single and multi agents scores and returns them
    
    Parameters
    ----------
    multi_agent_elo_score
    single_agent_elo_score
    """
    ret = evaluate_agent(
        {'q_value_pretrained': agent_path},
        initial_elo_ranking.index.values.tolist()[:5],
        n_matches=n_matches, single_agent=False, max_workers=20)
    table_multi = compute_elo_ranking(*ret)
    ret = evaluate_agent(
        {'q_value_pretrained': agent_path},
        initial_elo_ranking.index.values.tolist()[:5],
        n_matches=n_matches, single_agent=True, max_workers=20)
    table_single = compute_elo_ranking(*ret)
    plt.close('all')
    clear_output()
    print(agent_path)
    print('Multi agent elo score: %i' % table_multi.loc['q_value_pretrained', 'ranking'])
    print('Single agent elo score: %i' % table_single.loc['q_value_pretrained', 'ranking'])
    return table_multi.loc['q_value_pretrained', 'ranking'], table_single.loc['q_value_pretrained', 'ranking']

In [12]:
def simple_model_evaluation(model_path, n_matches=500):
    model_path = os.path.realpath(model_path)
    text = """
import os
from hungry_geese.agents import QValueSafeAgent
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
model = tf.keras.models.load_model('model_path', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
q_value_agent = QValueSafeAgent(model)
def agent(obs, config):
    return q_value_agent(obs, config)
    """
    text = text.replace('model_path', model_path)
    with tempfile.TemporaryDirectory() as tempdir:
        agent_filepath = os.path.join(tempdir, 'agent.py')
        with open(agent_filepath, 'w') as f:
            f.write(text)
        output = simple_agent_evaluation(agent_filepath, n_matches=n_matches)
    return output

### Visualization

In [13]:
def plot_q_value(agent):
    values = np.array(agent.q_values)
    for idx, action in enumerate(ACTIONS):
        plt.plot(values[:, idx], label=action, alpha=0.6, marker='o')
    plt.legend(loc=0)
    plt.grid()

In [14]:
COLOR_TO_INDEX = dict(white=0, blue=1, green=2, red=3)

In [15]:
def save_html_game(agents, output_folder):
    """
    Plays a game, saves to file and opens it on google chrome
    
    
    Order of the agents
    
    - 0 white
    - 1 blue
    - 2 green
    - 3 red
    """
    os.makedirs(output_folder, exist_ok=True)
    env = make('hungry_geese')
    ret = env.run(agents);
    html_content = env.render(mode="html")
    filepath = os.path.join(output_folder, '%s.html' % get_timestamp())
    with open(filepath, 'w') as f:
        f.write(html_content)
    print(filepath)
    os.system('google-chrome "%s"' % os.path.realpath(filepath))

In [16]:
def save_html_games(agents, output_folder, n_games=10):
    os.makedirs(output_folder, exist_ok=True)
    env = make('hungry_geese')
    for game_idx in tqdm(range(n_games)):
        ret = env.run(agents);
        html_content = env.render(mode="html")
        filepath = os.path.join(output_folder, '%02d.html' % game_idx)
        with open(filepath, 'w') as f:
            f.write(html_content)

## Visualizing games 

### 07_x64_discount_factor_1

In [17]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/36_deep_learning_longer_trains/07_x64_discount_factor_1/epoch_0187.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [QValueAgent(model) for idx in range(4)]

In [18]:
[agent.reset() for agent in agents]; save_html_game(agents, '../data/saved_games/04_discount_factor_1')

../data/saved_games/04_discount_factor_1/2021_04_11_17_37_45.html


It is strange that at epoch 33 white does not eat food.

In [32]:
[agent.reset() for agent in agents]; save_html_game(agents, '../data/saved_games/04_discount_factor_1')

../data/saved_games/04_discount_factor_1/2021_04_11_17_54_25.html


At epoch 117 prediction of white is totally wrong.

In [45]:
[agent.reset() for agent in agents]; save_html_game(agents, '../data/saved_games/04_discount_factor_1')

../data/saved_games/04_discount_factor_1/2021_04_11_18_03_20.html


So we have seen that the model has not learned a good enough reward function, sometimes ignores food and sometimes does not know that death is ahead.

In [50]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/36_deep_learning_longer_trains/07_x64_discount_factor_1/epoch_0187.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [SoftmaxAgent(model, scale=4) for idx in range(4)]

In [52]:
env = make('hungry_geese')
steps = []
for _ in tqdm(range(10)):
    [agent.reset() for agent in agents]
    env.run(agents)
    steps.extend([len(agent.state.rewards) for agent in agents])

  0%|          | 0/10 [00:00<?, ?it/s]

In [55]:
print(steps)

[198, 185, 198, 198, 49, 198, 198, 198, 198, 198, 198, 198, 198, 198, 178, 198, 198, 198, 198, 198, 198, 198, 196, 198, 198, 198, 198, 198, 198, 198, 198, 198, 195, 198, 198, 198, 198, 179, 198, 198]


In [54]:
np.mean(steps)+2

194.85

We can see that the metrics provided by train script seem to be real.

In [46]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/36_deep_learning_longer_trains/07_x64_discount_factor_1/epoch_0187.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [SoftmaxAgent(model, scale=8) for idx in range(4)]
env = make('hungry_geese', debug=False)
steps = []
for _ in tqdm(range(10)):
    [agent.reset() for agent in agents]
    env.run(agents)
    steps.extend([len(agent.state.rewards) for agent in agents])
print(np.mean(steps)+2)
print(steps)

  0%|          | 0/10 [00:00<?, ?it/s]

182.65
[160, 198, 198, 198, 198, 198, 198, 188, 135, 198, 198, 198, 198, 39, 198, 153, 198, 198, 172, 197, 198, 198, 198, 198, 198, 198, 167, 167, 86, 148, 198, 198, 198, 198, 157, 198, 195, 198, 198, 114]


In [49]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/36_deep_learning_longer_trains/07_x64_discount_factor_1/epoch_0187.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [QValueAgent(model) for idx in range(4)]
env = make('hungry_geese', debug=False)
steps = []
for _ in tqdm(range(10)):
    [agent.reset() for agent in agents]
    env.run(agents)
    steps.extend([len(agent.state.rewards) for agent in agents])
print(np.mean(steps)+2)
print(steps)

  0%|          | 0/10 [00:00<?, ?it/s]

176.8
[198, 127, 198, 198, 198, 198, 9, 153, 162, 151, 198, 198, 171, 198, 126, 198, 137, 198, 198, 198, 198, 147, 147, 198, 168, 198, 130, 198, 169, 198, 198, 147, 130, 198, 186, 198, 198, 198, 198, 178]


In [50]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/36_deep_learning_longer_trains/07_x64_discount_factor_1/epoch_0187.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [QValueSafeAgent(model) for idx in range(4)]
env = make('hungry_geese', debug=False)
steps = []
for _ in tqdm(range(10)):
    [agent.reset() for agent in agents]
    env.run(agents)
    steps.extend([len(agent.state.rewards) for agent in agents])
print(np.mean(steps)+2)
print(steps)

  0%|          | 0/10 [00:00<?, ?it/s]

174.525
[198, 147, 198, 130, 198, 173, 198, 198, 198, 194, 198, 166, 129, 170, 198, 198, 198, 139, 138, 198, 198, 92, 92, 198, 113, 198, 198, 113, 193, 194, 153, 194, 143, 198, 122, 198, 198, 153, 191, 198]


Let's see games against other agents.

In [ ]:
[agent.reset() for agent in agents]; save_html_game(agents, '../data/saved_games/04_discount_factor_1')

## Populating the agent pool

### Adding new agents

In [18]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/classic.py', n_matches=500)

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/q_value_safe_template.py
Multi agent elo score: 1365
Single agent elo score: 1393


(1365, 1393)

In [17]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/ram.py', n_matches=500)

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/ram.py
Multi agent elo score: 1463
Single agent elo score: 1471


(1463, 1471)

In [18]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/iceberg.py', n_matches=500)

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/iceberg.py
Multi agent elo score: 1548
Single agent elo score: 1551


(1548, 1551)

In [17]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py', n_matches=500)

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py
Multi agent elo score: 1602
Single agent elo score: 1565


(1602, 1565)

In [17]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/balerion.py', n_matches=500)

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/balerion.py
Multi agent elo score: 1638
Single agent elo score: 1616


(1638, 1616)

### Does having more good agents stabilize scores?

Let's evaluate a model with and without the new agents to see if there is any improvement on elo score stability. I will be using 100 matches.

In [17]:
initial_elo_ranking = initial_elo_ranking[1:]
initial_elo_ranking

,ranking
model,
iceberg,1548
wallbreaker,1515
ram,1463
quantum,1398
clasic,1365
boilergoose,1269
besthoarder,1227
crazy_goose,1186
risk_averse_goose,1176


In [18]:
new_scores = [simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py', n_matches=100) for _ in range(10)]

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py
Multi agent elo score: 1587
Single agent elo score: 1591


In [19]:
initial_elo_ranking = initial_elo_ranking.loc[['wallbreaker', 'quantum', 'boilergoose', 'besthoarder', 'crazy_goose']]
initial_elo_ranking

,ranking
model,
wallbreaker,1515
quantum,1398
boilergoose,1269
besthoarder,1227
crazy_goose,1186


In [20]:
old_scores = [simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py', n_matches=100) for _ in range(10)]

/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/icedragon.py
Multi agent elo score: 1584
Single agent elo score: 1528


In [24]:
new_scores = np.array(new_scores)
old_scores = np.array(old_scores)

In [26]:
np.mean(new_scores[:, 0]), np.std(new_scores[:, 0])

(1586.8, 21.738445206591937)

In [27]:
np.mean(old_scores[:, 0]), np.std(old_scores[:, 0])

(1559.9, 27.49709075520536)

In [ ]:
TODO: install seaborn to better visualize this and repeat with more runs.

In [25]:
new_scores.shape

(10, 2)

In [22]:
old_scores

[(1575, 1548),
 (1524, 1539),
 (1599, 1543),
 (1572, 1582),
 (1542, 1617),
 (1511, 1558),
 (1543, 1568),
 (1588, 1561),
 (1561, 1580),
 (1584, 1528)]

In [21]:
new_scores

[(1614, 1544),
 (1633, 1634),
 (1575, 1580),
 (1563, 1629),
 (1595, 1631),
 (1595, 1533),
 (1572, 1569),
 (1565, 1556),
 (1569, 1603),
 (1587, 1591)]

## Useful snippets

In [ ]:
simple_model_evaluation('/mnt/hdd0/Kaggle/hungry_geese/models/32_automated_iteration/13_continue_with_smaller_lr/epoch_0447.h5')

In [ ]:
simple_agent_evaluation('/mnt/hdd0/MEGA/AI/22 Kaggle/hungry_geese/data/agents/wallbreaker_data_augmentation.py', n_matches=500)

In [ ]:
model = keras.models.load_model('/mnt/hdd0/Kaggle/hungry_geese/models/32_automated_iteration/11_continue_with_scale_8/epoch_0347.h5', compile=False)
model = tf.keras.models.Model(inputs=model.inputs[:2], outputs=model.layers[-3].output)
agents = [QValueAgent(model) for idx in range(4)]

[agent.reset() for agent in agents]; save_html_game(agents, '../data/saved_games/03_clip_len_reward_analysis')
plot_q_value(agents[COLOR_TO_INDEX['white']]); plt.xlim(100)

In [ ]:
agent_base = '../data/agents/quantum/epsilon_quantum_010.py'
best_5_agents = [agents[name] for name in initial_elo_ranking.index.values[:4]]
best_5_agents += ['/mnt/hdd0/Kaggle/hungry_geese/models/25_ladder_of_agents/04_epsilon010_vs_best5_reward_-4_4_x128_80000_pretrained/q_value_agent.py']
sampling_agent_func = lambda: [agent_base] + np.random.choice(best_5_agents, 3).tolist()

for n_matches in [8000, 39000, 40000]:
    output_path = '/mnt/hdd0/Kaggle/hungry_geese/games/%s_vs_best5_it2_matches_%i.json' % (os.path.splitext(os.path.basename(agent_base))[0], n_matches)
    play_matches_in_parallel_and_save_history(agents=sampling_agent_func, n_matches=n_matches, output_path=output_path)
    
reward_name = 'ranking_reward_-4_4'

saved_games_paths = ['/mnt/hdd0/Kaggle/hungry_geese/games/%s_vs_best5_it2_matches_%i.json' % (os.path.splitext(os.path.basename(agent_base))[0], n_matches) for n_matches in [8000]]
output_path = '/mnt/hdd0/Kaggle/hungry_geese/data/%s/%s_vs_best5_it2_matches_%i.npz' % (reward_name, os.path.splitext(os.path.basename(agent_base))[0], 8000)
create_train_data_from_saved_matches(saved_games_paths, reward_name, output_path, agent_idx_range=[0])

saved_games_paths = ['/mnt/hdd0/Kaggle/hungry_geese/games/%s_vs_best5_it2_matches_%i.json' % (os.path.splitext(os.path.basename(agent_base))[0], n_matches) for n_matches in [39000, 40000]]
output_path = '/mnt/hdd0/Kaggle/hungry_geese/data/%s/%s_vs_best5_it2_matches_%i.npz' % (reward_name, os.path.splitext(os.path.basename(agent_base))[0], 80000)
create_train_data_from_saved_matches(saved_games_paths, reward_name, output_path, agent_idx_range=[0])

In [ ]:
agent_names = initial_elo_ranking.index.values.tolist()[:4]
output_folder = '../data/saved_games/02_quantum'
save_html_games([agents[name] for name in agent_names], output_folder, n_games=10)

In [ ]:
ret = evaluate_agent(
    {'q_value_pretrained': '/mnt/hdd0/Kaggle/hungry_geese/models/25_ladder_of_agents/01_epsilon025_reward_-4_4_x128_30000_pretrained/q_value_agent.py'},
    initial_elo_ranking.index.values.tolist()[:5],
    n_matches=500, single_agent=False, max_workers=20)
compute_elo_ranking(*ret)

In [ ]:
agent_filepaths = sorted(glob.glob('/mnt/hdd0/Kaggle/hungry_geese/models/28_architecture_study/*/q_value_agent.py'))
print(agent_filepaths)
df = pd.DataFrame()
for agent_filepath in agent_filepaths:
    print(df)
    ret = simple_agent_evaluation(agent_filepath, n_matches=500)
    name = os.path.basename(os.path.dirname(agent_filepath))
    df.loc[name, 'multi'] = ret[0]
    df.loc[name, 'single'] = ret[1]
df

## Summary